# Generate Embeddings

Using Facebook RAG.

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import faiss
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
import torch

C:\Users\test\PycharmProjects\cse227-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the RAG tokenizer and model
tokenizer = RagTokenizer.from_pretrained('facebook/rag-token-nq')
model = RagTokenForGeneration.from_pretrained('facebook/rag-token-nq')

# Load the retriever (which needs a corpus or index)
#retriever = RagRetriever.from_pretrained('facebook/rag-token-nq', index_name="exact", use_dummy_dataset=True)

C:\Users\test\PycharmProjects\cse227-project\.venv\Lib\site-packages\transformers\models\bart\configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is 

In [3]:
def get_inputs(file_path):
    read_inputs = []
    with open(file_path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            clean_line = line.strip()
            if not clean_line:
                continue
            read_inputs.append(clean_line)
    return read_inputs

In [4]:
# Input queries
queries = get_inputs('../data-input/questions.txt')

# Tokenize the input query
inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="pt")

In [5]:
# Get the embeddings of the query
with torch.no_grad():
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    embeddings = model.question_encoder(input_ids, attention_mask=attention_mask)[0]

In [6]:
custom_docs = get_inputs('../data-input/documents.txt')
docs_input = tokenizer(custom_docs, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    docs_input_ids = docs_input['input_ids']
    docs_attention_mask = docs_input['attention_mask']
    docs_embeddings = model.question_encoder(docs_input_ids, attention_mask=docs_attention_mask)[0]

In [7]:
dimension = docs_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(docs_embeddings)

In [8]:
k = 3
distances, indices = index.search(embeddings, k)

In [9]:
state = {
    'questions': queries,
    'question_embeddings': embeddings,
    'documents': custom_docs,
    'document_embeddings': docs_embeddings,
}

In [10]:
torch.save(state, '../data-output/embeddings.pt')